1.RunnablePassThrough

2.RunnableLambda

3.Runnable parallel 

      - itemgetter 

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI(model = "gpt-3.5-turbo-0125")

**RunnablePassthrough**

- it does not do anything to the input data.
- Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original without any modification. 

In [4]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()


In [5]:
chain.invoke("Bappy")

'Bappy'

**RunnableLambda**
- To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
- Let's define a very function to create Russian lastnames.


In [6]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [7]:
russian_lastname("Hassii")

'Hassiiovich'

In [9]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [10]:
chain.invoke("Hassii")

'Hassiiovich'

RunnableParallel 
- We will use RunnableParallel() for running tasks in parallel.
- This is probably the most important and most useful Runnable from Langchain.
- In the following chain, RunnableParallel is going to run these two tasks in parallel:
       - operation_a will use RunnablePassthrough.
       - operation_b will use RunnableLambda with the russian_lastname function

In [12]:
from langchain_core.runnables import RunnableParallel 

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [13]:
chain.invoke("HASSII")

{'operation_a': 'HASSII', 'operation_b': 'HASSIIovich'}

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [15]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [16]:
def russian_lastname_from_dictionary(person):
    return person['name'] + "ovich"

In [17]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_b": RunnablePassthrough(),
    }
)  | prompt | model | output_parser

In [18]:
chain.invoke({
              "name1": "Jordam",
              "name": "Abrahm"
              })

'One interesting fact about Roman Abramovich is that he is known for his extravagant lifestyle and love for luxury. He is the owner of several luxury properties, yachts, and private jets, and has been known to host lavish parties and events. Additionally, he is a prominent art collector and has one of the largest private art collections in the world. Abramovich is also known for his philanthropy, having donated millions of dollars to various charitable causes and organizations.'

let's see a more advanced use of RunnableParallel 

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Define the embedding model
embedding_model = OpenAIEmbeddings()

# Create the vectorstore with the embedding model
vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python Programming, etc in English"],
    embedding_model
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt   
    | model
    | StrOutputParser()
    )   

retrieval_chain.invoke("what is dswithbappy?")


'dswithbappy is a platform that focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python Programming, etc in English.'

In [29]:
from langchain_core.output_parsers import StrOutputParser 
from langchain.vectorstores import FAISS 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel 
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings 
from operator import itemgetter

# Define the embedding model
model = ChatOpenAI(model="gpt-3.5-turbo-0125")


# Create the vectorstore with the embedding model

vector_db = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python Programming, etc in English"],
    embedding_model
)


retriever = vector_db.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt 
    | model 
    | StrOutputParser()

)

chain.invoke({"question": "what is dswithbappy?", "language": "Pirate English"})

"Arrr, dswithbappy be a scallywag that focuses on providin' content on Data Science, AI, ML, DL, CV, NLP, Python Programmin', etc in English. Aye, me hearties!"